In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
import joblib

def kategoriser_tidspunkt(tid: str) -> str:
    hour = int(tid.split(":")[0])
    if 7 <= hour < 9:
        return "morgenrush 07:00-08:59"
    elif 9 <= hour < 15:
        return "formiddag 09:00-14:59"
    elif 15 <= hour < 18:
        return "ettermiddagrush 15:00-17:59"
    elif 18 <= hour < 24:
        return "kveld 18:00-23:59"
    else:
        return "natt 00:00-06:59"

def kategoriser_temperatur(temp: float) -> str:
    if temp < -5:
        return "kaldere enn - 5 grader"
    elif -5 <= temp <= 5:
        return "-5 t.o.m. 5 grader"
    else:
        return "varmere enn 5 grader"

def rens_kommune(tekst: str) -> str:
    # Fjerner kommune-ID og gjør til lowercase
    return tekst.split(" - ")[-1].strip().lower()
    
# Last inn og tilpass data
df = pd.read_csv("../data/processed/Ulykke_med_time.csv", encoding="ISO-8859-1", sep=";")
df.columns = df.columns.str.strip()
df["Kommune"] = df["Kommune"].apply(rens_kommune)

# Smelt kolonner til én rad per tidspunkt
melted = df.melt(id_vars=["Kommune", "Måned", "År"], var_name="Tidspunkt", value_name="Antall")
melted = melted.dropna(subset=["Antall", "Kommune", "Måned", "År", "Tidspunkt"])
melted["Tidspunkt"] = melted["Tidspunkt"].str.lower()

melted["Risiko"] = pd.cut(melted["Antall"], bins=[-1, 0, 2, float("inf")], labels=["lav", "middels", "høy"])

# Modell
X = melted[["Kommune", "Måned", "År", "Tidspunkt"]]
y = melted["Risiko"]

pipeline_tid = Pipeline([
    ("prep", ColumnTransformer([
        ("cat", OneHotEncoder(handle_unknown="ignore"), ["Kommune", "Måned", "Tidspunkt"]),
        ("num", "passthrough", ["År"])
    ])),
    ("clf", RandomForestClassifier(n_estimators=50, random_state=42))
])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
pipeline_tid.fit(X_train, y_train)
joblib.dump(pipeline_tid, "modell_tidspunkt_pipeline.pkl")

def prediker_tid(kommune, måned, år, tid_klokkeslett):
    kat_tid = kategoriser_tidspunkt(tid_klokkeslett)
    kommune_renset = rens_kommune(kommune)
    input_df = pd.DataFrame([[kommune_renset, måned, år, kat_tid]],
                            columns=["Kommune", "Måned", "År", "Tidspunkt"])
    return pipeline_tid.predict(input_df)[0]




In [11]:
tidspunkt_kategori = kategoriser_tidspunkt("08:30")
resultat_tid = prediker_tidspunktmodell("Oslo", "Januar", 2023, tidspunkt_kategori)

In [12]:
resultat_tid

'lav'